### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import json

### Load Data

In [30]:
with open("../data/heroes2id.json", "r") as f:
  HERO2ID = json.load(f)

with open("../data/heroes2role.json", "r") as f:
  HERO2ROLE = json.load(f)

with open("../data/patch_version2id.json", "r") as f:
  PATCH2ID = json.load(f)
  
with open("../data/team2id.json", "r") as f:
  TEAM2ID = json.load(f)


TOTAL_HEROES = len(HERO2ID)
BAN_STEPS = [1, 2, 3, 4, 5, 6, 13, 14, 15, 16]
TEAM1_STEPS = [1, 3, 5, 7, 10, 11, 14, 16, 18, 19]
ROLE2ID = {"goldlane": 0, "jungle": 1, "roam": 2, "midlane": 3, "explane": 4}

In [37]:
draft_steps_df = pd.read_csv("../data/processed/all_season_draft_steps.csv")
draft_steps_df.head()

,game_id,date,team_1,team_2,game_number,team_winner,patch_version,season,step_1,step_2,...,step_11,step_12,step_13,step_14,step_15,step_16,step_17,step_18,step_19,step_20
0,1,9-Aug-2024,tlid,fnoc,1,fnoc,1.9.06,14,fanny,chip,...,moskov,arlott,x.borg,joy,hylos,nolan,chou,julian,minotaur,harith
1,2,9-Aug-2024,fnoc,tlid,2,fnoc,1.9.06,14,zhuxin,chip,...,harith,arlott,terizla,hayabusa,hylos,luo yi,vexana,minotaur,thamuz,joy
2,3,9-Aug-2024,dewa,evos,1,dewa,1.9.06,14,terizla,chip,...,claude,vexana,minotaur,hayabusa,carmilla,alpha,ruby,julian,hylos,nolan
3,4,9-Aug-2024,dewa,evos,2,evos,1.9.06,14,terizla,chip,...,moskov,hylos,fanny,alpha,joy,hayabusa,guinevere,julian,x.borg,claude
4,5,9-Aug-2024,dewa,evos,3,dewa,1.9.06,14,zhuxin,chip,...,moskov,x.borg,fanny,luo yi,hayabusa,minotaur,guinevere,julian,paquito,novaria


### Functions

In [38]:
def hero_to_id(df: pd.DataFrame):
  step_cols = [f"step_{i}" for i in range(1, 21)]
  df[step_cols] = df[step_cols].replace(HERO2ID)
  return df

draft_steps_id_df = hero_to_id(draft_steps_df.copy())
draft_steps_id_df.head()

C:\Users\segav\AppData\Local\Temp\ipykernel_1752\3172899559.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[step_cols] = df[step_cols].replace(HERO2ID)


,game_id,date,team_1,team_2,game_number,team_winner,patch_version,season,step_1,step_2,...,step_11,step_12,step_13,step_14,step_15,step_16,step_17,step_18,step_19,step_20
0,1,9-Aug-2024,tlid,fnoc,1,fnoc,1.9.06,14,16,123,...,30,119,82,117,48,121,25,115,18,72
1,2,9-Aug-2024,fnoc,tlid,2,fnoc,1.9.06,14,124,123,...,72,119,81,20,48,95,37,18,71,117
2,3,9-Aug-2024,dewa,evos,1,dewa,1.9.06,14,81,123,...,64,37,18,20,91,27,28,115,48,121
3,4,9-Aug-2024,dewa,evos,2,evos,1.9.06,14,81,123,...,30,48,16,27,117,20,79,115,82,64
4,5,9-Aug-2024,dewa,evos,3,dewa,1.9.06,14,124,123,...,30,82,16,95,20,18,79,115,102,118


In [26]:
def game_info(draft_steps_df: pd.DataFrame):
  # patch version
  patch_version = draft_steps_df["patch_version"].to_numpy()
  patch_version_encoded = [PATCH2ID[pv] for pv in patch_version]
  feat_patch_version = [[pv] for pv in patch_version_encoded for _ in range(14)]
  # team 1
  team_1 = draft_steps_df["team_1"].to_numpy()
  team_1_encoded = [TEAM2ID[t] for t in team_1]
  feat_team_1 = [[t] for t in team_1_encoded for _ in range(14)]
  # team 2
  team_2 = draft_steps_df["team_2"].to_numpy()
  team_2_encoded = [TEAM2ID[t] for t in team_2]
  feat_team_2 = [[t] for t in team_2_encoded for _ in range(14)]

  return np.concatenate([feat_patch_version, feat_team_1, feat_team_2], axis=1)

game_info_feat = game_info(draft_steps_id_df)
print("game info shape:", game_info_feat.shape)
print("game info:", game_info_feat[0])

game info shape: (5992, 3)
game info: [0 8 4]


In [25]:
def draft_state(draft_steps_df: pd.DataFrame):
  all_state, steps, labels = [], [], []

  for _, row in draft_steps_df.iterrows():
    for step in range(7, 21):
      state = np.zeros(TOTAL_HEROES)
      for i in range(1, step):
        hero = row[f"step_{i}"]
        state[hero] = 1
      label = row[f"step_{step}"]
      labels.append(label)
      steps.append([step])
      all_state.append(state)

  return np.array(all_state), np.array(steps), np.array(labels)

draft_state_feat, step_feat, labels = draft_state(draft_steps_id_df)
print("draft state shape:", draft_state_feat.shape)
print("draft state:", draft_state_feat[0])
print("step shape:", step_feat.shape)
print("step:", step_feat[0])
print("labels shape:", labels.shape)
print("labels:", labels[0])

draft state shape: (5992, 128)
draft state: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0.]
step shape: (5992, 1)
step: [7]
labels shape: (5992,)
labels: 95


In [29]:
def hero_stats(draft_steps_df: pd.DataFrame, window_size: int = 0):
  history_picks, history_bans = [], []
  pickrate, banrate = [], []

  for _, row in draft_steps_df.iterrows():
    picks_window = history_picks if window_size == 0 else history_picks[-window_size:]
    bans_window = history_bans if window_size == 0 else history_bans[-window_size:]

    # hitung total pick & ban dari window
    pick_count = np.sum(picks_window, axis=0) if picks_window else np.zeros(TOTAL_HEROES)
    ban_count = np.sum(bans_window, axis=0) if bans_window else np.zeros(TOTAL_HEROES)
    window_len = len(picks_window)

    pickrate_game = pick_count / window_len if window_len > 0 else np.zeros(TOTAL_HEROES)
    banrate_game = ban_count / window_len if window_len > 0 else np.zeros(TOTAL_HEROES)

    for _ in range(7, 21):
      pickrate.append(pickrate_game.copy())
      banrate.append(banrate_game.copy())

    game_pick, game_ban = np.zeros(TOTAL_HEROES), np.zeros(TOTAL_HEROES)

    for step in range(1, 21):
      hero = row[f"step_{step}"]
      if step in BAN_STEPS:
        game_ban[hero] += 1
      else:
        game_pick[hero] += 1

    history_picks.append(game_pick)
    history_bans.append(game_ban)

  return np.array(pickrate), np.array(banrate)

pickrate_feat, banrate_feat = hero_stats(draft_steps_id_df, window_size=20)
print("pickrate shape:", pickrate_feat.shape)
print("pickrate:", pickrate_feat[-1])
print("banrate shape:", banrate_feat.shape)
print("banrate:", banrate_feat[-1])

pickrate shape: (5992, 128)
pickrate: [0.   0.   0.   0.   0.   0.2  0.   0.   0.   0.05 0.   0.05 0.   0.
 0.   0.   0.   0.   0.   0.   0.2  0.   0.   0.   0.   0.1  0.   0.15
 0.05 0.15 0.05 0.   0.   0.   0.1  0.   0.   0.05 0.   0.05 0.45 0.
 0.25 0.   0.   0.   0.1  0.   0.25 0.   0.   0.55 0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.05 0.1  0.   0.05 0.   0.05 0.   0.05 0.
 0.2  0.   0.7  0.   0.   0.05 0.4  0.   0.5  0.   0.   0.1  0.   0.3
 0.   0.   0.35 0.1  0.35 0.   0.   0.   0.   0.   0.   0.4  0.1  0.1
 0.   0.   0.2  0.   0.   0.3  0.   0.55 0.   0.   0.   0.2  0.05 0.
 0.   0.   0.   0.   0.2  0.3  0.   0.   0.   0.   0.25 0.15 0.25 0.1
 0.15 0.55]
banrate shape: (5992, 128)
banrate: [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.05 0.   0.   0.   0.
 0.   0.   0.9  0.   0.   0.   0.65 0.   0.   0.   0.   0.05 0.   0.
 0.   0.15 0.05 0.   0.   0.   0.2  0.   0.   0.   0.   0.   0.05 0.
 0.1  0.   0.   0.   0.   0.   0.05 0.   0.   0.2  0.   0.   0.   0.
 0.   0.   0

In [39]:
def role_state(draft_steps_df: pd.DataFrame):
  roles_picked_team1, roles_picked_team2 = [], []

  for _, row in draft_steps_df.iterrows():
    for step in range(7, 21):
      roles_team1, roles_team2 = np.zeros(5), np.zeros(5)
      for i in range(7, step):
        hero = row[f"step_{i}"]
        roles = HERO2ROLE[hero]
        roles_id = [ROLE2ID[role] for role in roles]
        if i not in BAN_STEPS:
          if i in TEAM1_STEPS:
            for role_id in roles_id: roles_team1[role_id] += 1
          else:
            for role_id in roles_id: roles_team2[role_id] += 1
        
      roles_picked_team1.append(roles_team1)
      roles_picked_team2.append(roles_team2)

  return np.array(roles_picked_team1), np.array(roles_picked_team2)

roles_team1_feat, roles_team2_feat = role_state(draft_steps_df)
print("roles team1 shape:", roles_team1_feat.shape)
print("roles team1:", roles_team1_feat[-1])
print("roles team2 shape:", roles_team2_feat.shape)
print("roles team2:", roles_team2_feat[-1])

roles team1 shape: (5992, 5)
roles team1: [1. 2. 2. 1. 2.]
roles team2 shape: (5992, 5)
roles team2: [1. 0. 1. 1. 2.]


### Export Features

In [46]:
features = np.concatenate([
  game_info_feat, draft_state_feat, pickrate_feat, banrate_feat,
  roles_team1_feat, roles_team2_feat, step_feat
], axis=1)
print("features shape:", features.shape)
print("labels shape:", labels.shape)

features shape: (5992, 398)
labels shape: (5992, 1)


In [50]:
labels = labels.reshape(-1, 1)
final = np.concatenate([features, labels], axis=1)
np.save("../data/features/gi_ds_pr20_br20_rt1_rt2_s.npy", final)

In [ ]:
# NAME FORMAT
# gi => game info
# ds => draft state
# pr{n} => pickrate_{window size}
# br{n} => banrate_{windowsize}
# rt1 => roles team 1
# rt2 => roles team 2
# s => steps